### Setup (only need to run once in our notebook instance)

In [64]:
!pip install pyathena

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [65]:
!pip install stop-words

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Load Modules

In [66]:
import boto3
import pandas as pd
import json
from sagemaker import get_execution_role
from my_utils import language_dict, text_clean_alphanum, \
                    det_tweet_language, translate_tweet, \
                    athena_cursor
import seaborn as sns

#the below are needed to connect to the athena interface
from pyathena.util import as_pandas
from pyathena import connect

from io import StringIO

# Syntax Analysis (part of speech tagging using Amazon Comprehend)

### Load df from athena (can also use stored .csv file)

In [67]:
cursor = connect(aws_access_key_id='access_key',
                 aws_secret_access_key='secret',
                 s3_staging_dir='s3://tweet-general/',
                 region_name='region').cursor()
#in the exectue() enter your athena command to view your chosen table/view
cursor.execute("SELECT * FROM coronavirus.tweet")
tweet_text= as_pandas(cursor)
tweet_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39513 entries, 0 to 39512
Data columns (total 17 columns):
tweet_id              39513 non-null object
user_id               39513 non-null object
created_at            39513 non-null object
created_at_date       39513 non-null object
source                39513 non-null object
truncated             39513 non-null bool
text                  39513 non-null object
coordinates           109 non-null object
place_type            39513 non-null object
place_name            39513 non-null object
place_full_name       39513 non-null object
place_country         39513 non-null object
place_country_code    39513 non-null object
reply_count           39513 non-null int64
retweet_count         39513 non-null int64
favorite_count        39513 non-null int64
hashtag               39513 non-null object
dtypes: bool(1), int64(3), object(13)
memory usage: 4.9+ MB


### Store the chosen columns from the df as the input df for the function syntax_analysis_chosen_tag

In [68]:
tweet_input_df = tweet_text[['tweet_id', 'user_id', 'text']] .copy()
tweet_input_df.head(5)

,tweet_id,user_id,text
0,1246162032161951745,887055386,"congrats to italian scientist andrea gambotto,..."
1,1240004685219725313,2630718094,"just think about this..\nin a few months time,..."
2,1240036280114860032,442446326,pump this into my veins
3,1240036282052509709,33794617,been knew ydqoeobwhq
4,1240036326126301184,46295482,y’all better sign them up for qctzonzyba onli...


### Define a function to write a filepath to S3

In [69]:
from io import StringIO
import boto3

def write_pd_s3_csv(df, bucket, filepath):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, filepath).put(Body=csv_buffer.getvalue())
    print("The data is successfully written to S3 path:", bucket+"/"+filepath)

### Define the function to use Amazon Comprehend to tag the tweet text and store the chosen tag and score

In [71]:
#input_tweet_df is a df including the tweet text we are analysing, stored from accessing the athena console, including 'tweet_id' and 'text' columns
#tag is the part of speech we are interesting in storing i.e. verbs, default set to VERB
#tag_score is the lowerbound of the partofspeech score for the tagging in comprehend, default set to 0.99
#output_df is a df where we store the results
#s3_bucket and the filename are your chosen bucket and filepath for the df to be saved as a .csv

def syntax_analysis_chosen_tag (tweet_input_df, word_tag = "VERB", tag_score = 0.99):

    comprehend = boto3.client(service_name='comprehend', region_name='eu-west-1')
    #create an empty dataframe to append our results to
    df_Syntax = pd.DataFrame(columns=['tweet_id', 'Score', 'Tag','Text'])

    #loop over the rows in the cleaned tweet table and analyse the syntax of the full tweet text
    for row in range(0,len(tweet_input_df):
        tweet_id = tweet_input_df.iloc[row]['tweet_id']
        text = tweet_input_df.iloc[row]['text']
        syntax_data = json.dumps(comprehend.detect_syntax(Text=text, LanguageCode='en'), sort_keys=True, indent=4)
        #store the json data as a python dictionary
        syntax_dic = json.loads(syntax_data)

        #create a for loop to loop through i syntaxtokens and return the score, tag, text
        for i in range(0,len(syntax_dic['SyntaxTokens'])):
        #only store if the sytanx tag is a verb
            if (syntax_dic['SyntaxTokens'][i]['PartOfSpeech']['Tag'] == word_tag and syntax_dic['SyntaxTokens'][i]['PartOfSpeech']['Score'] > tag_score):
                Score = syntax_dic['SyntaxTokens'][i]['PartOfSpeech']['Score']
                Tag = syntax_dic['SyntaxTokens'][i]['PartOfSpeech']['Tag']
                Text = syntax_dic['SyntaxTokens'][i]['Text']
                df_Syntax = df_Syntax.append({'tweet_id' : tweet_id, 'Score' : Score, 'Tag' : Tag, 'Text' : Text}, ignore_index = True)

    return df_Syntax

### To run the syntax analysis run the below function with your chosen variables (defaults have been set for most - change the filename if run more than once) sotre it to a df and then export this df to S3

In [52]:
verb_syntax_date = syntax_analysis_chosen_tag (tweet_input_df, word_tag = "VERB", tag_score = 0.99)
verb_syntax_date.head(5)

In [46]:
# Write to the chosen df to S3
s3_bucket =  'virus-syntax-analysis'  
file_path = 'verb_date.csv'
write_pd_s3_csv(verb_syntax_date, s3_bucket, file_path)

The data is successfully written to S3 path: virus-syntax-analysis/noun_0406.csv


In [ ]:
noun_syntax_date = syntax_analysis_chosen_tag (tweet_input_df, word_tag = "NOUN", tag_score = 0.99)
noun_syntax_date.head(5)

In [ ]:
# Write to the chosen df to S3
s3_bucket =  'virus-syntax-analysis'  
file_path = 'noun_date.csv'
write_pd_s3_csv(noun_syntax_date, s3_bucket, file_path)